In [1]:
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env

import rllib_setup

ray.init()
env_name = "VJS"
register_env(
    env_name,
    lambda config: rllib_setup.get_env_continuous(),
)
test_env = rllib_setup.get_env_continuous()


def policies(agent_ids):
    obs_space = test_env.observation_space
    act_space = test_env.action_space
    return {
        str(i): (
            None,
            obs_space,
            act_space,
            {}
            # config=config.overrides(agent_id=int(i[8:])),
        )
        for i in agent_ids
    }


config = (
    SACConfig().rollouts(num_rollout_workers=10, rollout_fragment_length=30)
    .training(lr=0.01)
    .resources(num_gpus=1)
    .multi_agent(
        policies=policies(test_env._agent_ids),
        policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
    )
    .environment(disable_env_checking=True)
)
config.batch_mode = "complete_episodes"
print(config.to_dict())
# Build a Algorithm object from the config and run one training iteration.
algo = config.build(env=env_name)

2023-09-29 09:56:24,736	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8269 
2023-09-29 09:56:26,371	INFO algorithm.py:536 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 1, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': True, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_envs_pe

2023-09-29 09:56:38,848	INFO trainable.py:173 -- Trainable.setup took 12.479 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [5]:
algo.train()['info']['learner']['Machine_0']

(raylet) [2023-09-29 10:25:28,905 E 234470 234470] (raylet) node_manager.cc:3069: 17 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: ae01304372a2c7300d1ab620833e4bf5a240b8299c4a019fb5c0cee1, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
2023-09-29 10:25:37,796	ERROR actor_manager.py:507 -- Ray error, taking actor 1 out of service. The actor died unexp

{'custom_metrics': {},
 'learner_stats': {'actor_loss': -0.03800068795681,
  'critic_loss': 2455.9326171875,
  'alpha_loss': -0.0005457288352772593,
  'alpha_value': 0.99898785,
  'log_alpha_value': -0.0010126906,
  'target_entropy': -1.0,
  'policy_t': -0.5082023739814758,
  'mean_q': 0.8757630586624146,
  'max_q': 1.0126042366027832,
  'min_q': 0.7055069804191589},
 'model': {},
 'num_grad_updates_lifetime': 4.0,
 'diff_num_grad_updates_vs_sampler_policy': 3.0,
 'td_error': array([3.3730578e+04, 1.1208012e+00, 9.2655194e-01, 7.4554664e-01,
        6.9908106e-01, 6.3477272e-01, 1.1870246e+00, 2.9447624e-01,
        5.3123008e+04, 1.4234613e+00, 1.2141159e+00, 3.7471911e-01,
        1.0757611e+00, 5.4920227e+04, 1.4119432e+00, 6.5867484e-01,
        9.2449862e-01, 9.9356091e-01, 1.2611229e+00, 2.6151027e+04,
        9.2048097e-01, 1.2381991e+00], dtype=float32),
 'mean_td_error': 7633.724609375}

(raylet) [2023-09-29 10:26:30,345 E 234470 234470] (raylet) node_manager.cc:3069: 18 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: ae01304372a2c7300d1ab620833e4bf5a240b8299c4a019fb5c0cee1, IP: 192.168.3.6) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.3.6`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-09-29 10:27:39,050 E 234470 234470] (raylet) node_manager.cc:3069: 16 Workers (tasks / actors) killed